In [1]:
import cv2
import torch
from PIL import Image
import cv2
import torch
from PIL import Image
import glob
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.image as mpimg
import os
from pyzbar.pyzbar import decode
from pyzbar import pyzbar
import cv2
import glob
from tqdm import tqdm
from barcode import EAN13
from barcode.writer import ImageWriter
import matplotlib.pyplot as plt

import cv2
import torch
from PIL import Image

import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


%matplotlib inline

C:\Users\sendr\anaconda3\envs\yolov5new\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [93]:
pwd

'D:\\Proj_OCR'

In [2]:
!pip install ipywidgets

  Using cached ipywidgets-7.7.0-py2.py3-none-any.whl (123 kB)
  Using cached jupyterlab_widgets-1.1.0-py3-none-any.whl (245 kB)
  Using cached widgetsnbextension-3.6.0-py2.py3-none-any.whl (1.6 MB)


In [3]:
#YOLO model loading - detect locations of labels
model = torch.hub.load('ultralytics/yolov5', 'custom', path='./best_sticker_OCR_All.pt')
model.iou = 0.1
print(type(model))

Using cache found in C:\Users\sendr/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2021-12-9 torch 1.8.2+cu111 CUDA:0 (GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
Model Summary: 224 layers, 7062001 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


<class 'models.common.AutoShape'>


In [4]:
files = glob.glob('./Sample_Images/*.png')
print(len(files))
another_files = glob.glob('./Sample_Images/*.jpg')

for file in another_files:
    files.append(file)
    
    
print(len(files))

0
7


In [91]:
class Sticker:
    def __init__(self, image = None):
        self.image = image
        
        #OCR purpose
        self.results = []
        self.Attributes = []
        self.Contents = []
        
        #Barcode purpose
        self.barcodeList = []
        
        

'''class Data:
    def __init__(self, Attribute, Content):
        self.Attribute = None
        self.Content = None'''
## OCR Purpose        
class Attribute:
    def __init__(self, start_point, end_point, middle_point, image):    
        self.start_point = start_point
        self.end_point = end_point
        self.middle_point = middle_point
        
        self.xmin = start_point[0]
        self.ymin = start_point[1]
        
        self.xmax = end_point[0]
        self.ymax = end_point[1]
        
        self.xmid = (xmin + xmax)//2
        self.ymid = (ymin + ymax)//2
        
        self.image = image[ymin:ymax,xmin:xmax]
        self.OCR = None
        self.WriteSpot = None
        #self.WriteSpot = (xmid, ymin)
        
class Content:
    def __init__(self, start_point, end_point, middle_point, image):       
        self.start_point = start_point
        self.end_point = end_point
        self.middle_point = middle_point
        
        self.xmin = None
        self.ymin = None
        
        self.xmax = None
        self.ymax = None
        
        self.xmid = None
        self.ymid = None
        
        self.image = None
        self.OCR = None
        self.WriteSpot = None
        #self.WriteSpot = (xmax, ymid)
        
# Barcode Purpose
class Barcode:
    def __init__(self,  barcodeContent, barcodeType, originAddress):
        self.barcodeContent = barcodeContent
        self.barcodeType = barcodeType
        self.boundingBox = None

class BoundingBox:
     def __init__(self, initPosition, finalPosition, theText):
         self.initPosition = initPosition
         self.finalPosition = finalPosition
         self.theText = theText
         self.color = (0,0,255)


In [83]:
#Operation on each stickers

sticker_list = []  # batch of images

#read each file of sticker images
for file in tqdm(files):
    sticker_list.append(Sticker(cv2.imread(file)))
    
#convert BGR to RGB
for i in range (len(sticker_list)):
    sticker_list[i].image = cv2.cvtColor(sticker_list[i].image, cv2.COLOR_BGR2RGB)
    
for i in range (len(sticker_list)):
    sticker_list[i].results = model(np.copy(sticker_list[i].image))    

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 43.86it/s]


In [84]:
a = sticker_list[0]
a.results

In [85]:
df = a.results.pandas().xyxy[0]
df

,xmin,ymin,xmax,ymax,confidence,class,name
0,231.387680,791.433105,345.996429,834.733032,0.801172,1,data
1,54.218082,170.539017,226.722321,231.844360,0.790500,0,attributes
2,42.042900,504.288849,221.389908,565.925842,0.766073,0,attributes
3,51.233402,382.124115,221.768021,435.113892,0.750557,0,attributes
4,60.140682,71.414818,215.920776,127.972366,0.747827,0,attributes
5,239.739304,668.682922,394.298981,712.286560,0.739107,1,data
6,234.533173,132.344162,460.120331,174.461136,0.737152,1,data
7,233.600037,232.648392,582.661865,278.225342,0.725224,1,data
8,231.769577,556.494324,358.932587,597.085022,0.716699,1,data
9,228.867035,444.031769,376.759857,484.910919,0.688982,1,data


In [86]:
df_att = df.loc[df['name'] == 'attributes' ]
df_cont = df.loc[df['name'] == 'data' ]

In [87]:
df_att

,xmin,ymin,xmax,ymax,confidence,class,name
1,54.218082,170.539017,226.722321,231.844360,0.790500,0,attributes
2,42.042900,504.288849,221.389908,565.925842,0.766073,0,attributes
3,51.233402,382.124115,221.768021,435.113892,0.750557,0,attributes
4,60.140682,71.414818,215.920776,127.972366,0.747827,0,attributes
11,106.461250,286.993652,207.055649,338.245361,0.652128,0,attributes
12,76.844986,729.789551,212.450394,792.615051,0.631310,0,attributes
13,108.699562,847.833252,633.452698,914.291199,0.617409,0,attributes
14,91.492081,612.314514,209.765488,672.126038,0.579938,0,attributes


In [88]:
for index in df_att.index:
    print(index)

1
2
3
4
11
12
13
14


In [89]:
df_cont

,xmin,ymin,xmax,ymax,confidence,class,name
0,231.387680,791.433105,345.996429,834.733032,0.801172,1,data
5,239.739304,668.682922,394.298981,712.286560,0.739107,1,data
6,234.533173,132.344162,460.120331,174.461136,0.737152,1,data
7,233.600037,232.648392,582.661865,278.225342,0.725224,1,data
8,231.769577,556.494324,358.932587,597.085022,0.716699,1,data
9,228.867035,444.031769,376.759857,484.910919,0.688982,1,data
10,232.030380,345.122223,508.774475,382.343689,0.660783,1,data
15,747.651062,404.941254,889.357849,472.153259,0.502310,1,data
16,135.451492,848.691101,608.439026,915.673279,0.260121,1,data


In [90]:
for index in df_cont.index:
    print(index)

0
5
6
7
8
9
10
15
16


In [35]:
img2 = np.copy(sticker_list[0].image)


(927, 972, 3)


In [59]:
img2 = np.copy(sticker_list[0].image)

xmin = int(df_att.iloc[1].xmin)
ymin = int(df_att.iloc[1].ymin)
xmax = int(df_att.iloc[1].xmax)
ymax = int(df_att.iloc[1].ymax)

xmid = (xmin+xmax)//2
ymid = (ymin+ymax)//2

window_name = 'Image.jpg'
start_point = (xmin, ymin)
end_point = (xmax, ymax)
middle_point = (xmid, ymid)

img2 = cv2.circle(img2, start_point, radius=10, color=(0, 0, 255), thickness=-10)
img2 = cv2.circle(img2, end_point, radius=10, color=(0, 0, 0), thickness=-10)
img2 = cv2.circle(img2, middle_point, radius=10, color=(0, 255, 0), thickness=-10)
  
# Displaying the image 
cv2.imwrite(window_name, img2) 

True

189.10843450253614


93.92017887546851


In [66]:
print(calculateDistance(middle_point, end_point))

95.1892851112981


In [ ]:
import torch

result_list = []

for img in tqdm(imgs):
    result_list.append(model2(img.image))
    
i = 0
for result in tqdm(result_list):
    result.save('./Detection_Results/Img'+str(i))
    i+=1